# Initialization

In [1]:
!rm -rf Deblur

In [2]:
!git clone https://github.com/JPBrito0528/Deblur

Cloning into 'Deblur'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 286 (delta 4), reused 17 (delta 2), pack-reused 267 (from 1)
Receiving objects: 100% (286/286), 397.38 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [3]:
!pip install pytorch-msssim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [4]:
!pip install numpy==1.24.3
!pip install imgaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 115.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 w

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 12.6 MB/s eta 0:00:00


In [1]:
pip install torchviz

In [2]:
import sys
sys.path.append('/content/Deblur')

from pytorch_msssim import ms_ssim
from google.colab.patches import cv2_imshow
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms.functional as TF
from torchviz import make_dot
import model
from model import common
import cv2 as cv
import numpy as np
import os
import torch.nn as nn
from tkinter import filedialog
import random
from math import ceil, floor
import pandas as pd
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

# Main

In [ ]:
# --------------------------
# Helper function to display images in Colab
def imshow_cv2(img, title="Image", resize_factor=0.5):
    """Display an image using matplotlib instead of cv.imshow."""
    # Convert BGR (OpenCV format) to RGB for display with matplotlib
    if len(img.shape) == 3 and img.shape[2] == 3:
        img_disp = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    else:
        img_disp = img
    # Resize image for display if desired
    if resize_factor != 1:
        img_disp = cv.resize(img_disp, (0, 0), fx=resize_factor, fy=resize_factor)

    plt.figure(figsize=(10, 6))
    plt.imshow(img_disp, cmap='gray' if len(img_disp.shape) == 2 else None)
    plt.title(title)
    plt.axis('off')
    plt.show()
# ----------------------------

DEBUG=False
TEST=False
TRAIN=True
TRAIN_TYPE = 4 # 0 = weighted ROI ; 1 = black inverted ROI ; 2 = traditional MSE ; 3 - binary output; 4 - SSIM
DNN_TYPE = 0 # 0 = EDSR ; 1 = UNET

IMG_WIDTH = int(1920/2)
IMG_HEIGHT = int(1080/2)
IMG_CHANNELS = 3
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

SOURCE_FOLDER = '/content/Deblur'
#SOURCE_FOLDER = 'C:/Users/guilherme.franco/Documents/GAF/Estagio_Rui_Brito'
training_history_filename = SOURCE_FOLDER+'/Results/log.csv'

seed=420
SPLIT = 0.895
EPOCHS = 500
BATCH_SIZE=4
LEARNING_RATE = 10e-5

### Setup Dataset ###############################################################################################

raw_df = pd.read_csv(SOURCE_FOLDER+'/Lens_Dataset/data.csv').sample(frac=1)  #(read and shuffle)

raw_size = int(len(raw_df))

train_df = raw_df.head(floor(int(raw_size*SPLIT)))
train_size = int(len(train_df))

val_df = raw_df.tail(ceil(int(raw_size*(1-SPLIT))))
val_size = int(len(val_df))

train_df = train_df.sample(frac=1)
val_df = val_df.sample(frac=1)

#################################################################################################################

def yield_training_batch(img_file_list, corners_list):

    batch_size = len(img_file_list)

    input_images = []
    output_images = []
    batch_corners = []

    for f in range(0,batch_size):

        # Read image from list and convert to array
        input_image_path = SOURCE_FOLDER+'/Lens_Dataset/INPUTS/'+str(img_file_list[f])+'.png'
        output_image_path = SOURCE_FOLDER+'/Lens_Dataset/OUTPUTS/'+str(img_file_list[f])+'.png'

        input_image = cv.imread(input_image_path)
        output_image = cv.imread(output_image_path)

        cornerTL_x = corners_list[f][0]
        cornerTL_y = corners_list[f][1]
        cornerTR_x = corners_list[f][2]
        cornerTR_y = corners_list[f][3]
        cornerBL_x = corners_list[f][4]
        cornerBL_y = corners_list[f][5]
        cornerBR_x = corners_list[f][6]
        cornerBR_y = corners_list[f][7]


        if DEBUG:
            debug_img = output_image.copy()
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),6,(255,255,255),-1)

            cv.imshow('PRE DEBUG OUTPT',  cv.resize(debug_img,(960,720)))


        #generate random vector to determine which augmentations to perform
        R = []
        for r in range(0,7):
            if DEBUG:
                R.append(0)
            else:
                rn = random.random()
                R.append(rn)

        #======ZOOM AND CROP OR NOT?======#

        if R[0] < 0.3:

            aspect_ratio = input_image.shape[1]/input_image.shape[0]
            target_ratio = IMG_WIDTH/IMG_HEIGHT
            height_center = int(input_image.shape[0]/2)
            width_center = int(input_image.shape[1]/2)

            if aspect_ratio > IMG_WIDTH/IMG_HEIGHT:
                target_width = int(input_image.shape[0]*target_ratio)
                input_image = input_image[0:input_image.shape[0], width_center-floor(target_width/2):width_center+floor(target_width/2)]
                output_image = output_image[0:input_image.shape[0], width_center-floor(target_width/2):width_center+floor(target_width/2)]

            elif aspect_ratio < IMG_WIDTH/IMG_HEIGHT:
                target_height = int(input_image.shape[1]*target_ratio)
                input_image = input_image[height_center-floor(target_height/2):height_center+floor(target_height/2), 0:input_image.shape[1]]
                output_image = output_image[height_center-floor(target_height/2):height_center+floor(target_height/2), 0:input_image.shape[1]]

            rn = random.uniform(1,1)    #how much to zoom in

            crop_w = floor(IMG_WIDTH*rn)    #how much to crop
            crop_h = floor(IMG_HEIGHT*rn)

            w_space = IMG_WIDTH - crop_w    #how much space is left
            h_space = IMG_HEIGHT - crop_h

            random_center_x = int(crop_w/2) + random.randint(0,w_space)     #center of the zoom/crop
            random_center_y = int(crop_h/2) + random.randint(0,h_space)

            #crops according to random parameters and resizes to desired input/output size
            input_image = input_image[random_center_y-int(crop_h/2):random_center_y+int(crop_h/2),random_center_x-int(crop_w/2):random_center_x+int(crop_w/2)]
            input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)
            output_image = output_image[random_center_y-int(crop_h/2):random_center_y+int(crop_h/2),random_center_x-int(crop_w/2):random_center_x+int(crop_w/2)]
            output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_NEAREST)


            # TODO: estas contas provavelmente estão mal, pelo que 'desabilitei o random crop e zoom'
            cornerTL_x = (cornerTL_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerTL_y = (cornerTL_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerTR_x = (cornerTR_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerTR_y = (cornerTR_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerBL_x = (cornerBL_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerBL_y = (cornerBL_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerBR_x = (cornerBR_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerBR_y = (cornerBR_y-(random_center_y - (IMG_HEIGHT/2)))*rn

        else:

            input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)
            output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)

        #======FLIP OR NOT?======#

        #Flip Horizontaly
        if R[1] < 0.5:
            input_image = cv.flip(input_image, 1)
            output_image = cv.flip(output_image, 1)
            cornerTL_x = (IMG_WIDTH) - cornerTL_x
            cornerTR_x = (IMG_WIDTH) - cornerTR_x
            cornerBL_x = (IMG_WIDTH) - cornerBL_x
            cornerBR_x = (IMG_WIDTH) - cornerBR_x

        #Flip Verticaly
        if R[2] < 0.5:
            input_image = cv.flip(input_image, 0)
            output_image = cv.flip(output_image, 0)
            cornerTL_y = (IMG_HEIGHT) - cornerTL_y
            cornerTR_y = (IMG_HEIGHT) - cornerTR_y
            cornerBL_y = (IMG_HEIGHT) - cornerBL_y
            cornerBR_y = (IMG_HEIGHT) - cornerBR_y

        #Color Shift
        if R[3] < 0.3:
            seq = iaa.Sequential([iaa.MultiplyHueAndSaturation((0.8, 1.2), per_channel=True)])
            input_image = seq.augment_image(input_image)
            output_image = seq.augment_image(output_image)

        #Random Simplex Noise Blobs
        # if R[4] < 0.3:
        #     seq = iaa.SimplexNoiseAlpha( first=iaa.Multiply(mul = (0.6,1.4),per_channel=True),per_channel=True)
        #     input_image = seq.augment_image(input_image)
        #     output_image = seq.augment_image(output_image)

        #Now we have to work in [0,1] instead of [0,255]
        input_image = input_image/255.0
        output_image = output_image/255.0

        # Gaussian Blur
        # if R[5] < 0.3:
        #     input_image = cv.GaussianBlur(input_image,(5,5),0)

        #Gaussian Noise
        # if R[6] < 0:#0.2:
        #     mean = 0
        #     var = 0.001
        #     sigma = var**0.5
        #     gauss = np.random.normal(mean,sigma,(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS))
        #     gauss = gauss.reshape(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
        #     input_image = input_image + gauss

        if DEBUG:
            debug_img = output_image.copy()
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),6,(255,255,255),-1)

            cv.imshow('DEBUG OUTPT',  cv.resize(debug_img,(960,720)))
            cv.waitKey()

        input_images.append(input_image)
        output_images.append(output_image)
        batch_corners.append((cornerTL_x,cornerTL_y,cornerTR_x,cornerTR_y,cornerBL_x,cornerBL_y,cornerBR_x,cornerBR_y))

    input_images_np = np.array(input_images)
    output_images_np = np.array(output_images)

    if len(input_images) == 0:
        # no valid images this batch
        return None, None, []

    # stack → (B, H, W, C)
    input_np  = np.stack(input_images,  axis=0).astype(np.float32)
    output_np = np.stack(output_images, axis=0).astype(np.float32)

    # one‑shot tensor conversion + permute → (B, C, H, W)
    input_tensor  = torch.from_numpy(input_np).permute(0, 3, 1, 2).cuda()
    output_tensor = torch.from_numpy(output_np).permute(0, 3, 1, 2).cuda()

    return input_tensor, output_tensor, batch_corners

def yield_training_batch_black_BG(img_file_list, corners_list):
    batch_size = len(img_file_list)
    input_images = []
    output_images = []
    batch_corners = []

    for f in range(batch_size):
        try:
            # Build paths and load images
            input_image_path = SOURCE_FOLDER + '/Lens_Dataset/INPUTS/' + str(img_file_list[f]) + '.png'
            output_image_path = SOURCE_FOLDER + '/Lens_Dataset/OUTPUTS/' + str(img_file_list[f]) + '.png'
            input_image = cv.imread(input_image_path)
            output_image = cv.imread(output_image_path)

            # Check if image loading failed; if so, skip this image.
            if input_image is None or output_image is None:
                print("Skipping image", img_file_list[f], "- failed to load one or both images.")
                continue

            # Extract corner coordinates
            cornerTL_x = corners_list[f][0]
            cornerTL_y = corners_list[f][1]
            cornerTR_x = corners_list[f][2]
            cornerTR_y = corners_list[f][3]
            cornerBL_x = corners_list[f][4]
            cornerBL_y = corners_list[f][5]
            cornerBR_x = corners_list[f][6]
            cornerBR_y = corners_list[f][7]

            # (Optional debugging display omitted if not needed)

            # Generate random vector for augmentation decisions
            R = []
            for r in range(7):
                # You can remove DEBUG-related conditions if not required.
                R.append(random.random())

            # ====== ZOOM AND CROP OR NOT? ====== #
            if R[0] < 0.3:
                aspect_ratio = input_image.shape[1] / input_image.shape[0]
                target_ratio = IMG_WIDTH / IMG_HEIGHT
                height_center = int(input_image.shape[0] / 2)
                width_center = int(input_image.shape[1] / 2)

                if aspect_ratio > target_ratio:
                    target_width = int(input_image.shape[0] * target_ratio)
                    input_image = input_image[:, width_center - floor(target_width / 2):width_center + floor(target_width / 2)]
                    output_image = output_image[:, width_center - floor(target_width / 2):width_center + floor(target_width / 2)]
                elif aspect_ratio < target_ratio:
                    target_height = int(input_image.shape[1] / target_ratio)
                    input_image = input_image[height_center - floor(target_height / 2):height_center + floor(target_height / 2), :]
                    output_image = output_image[height_center - floor(target_height / 2):height_center + floor(target_height / 2), :]

                rn = random.uniform(1, 1)  # how much to zoom in
                crop_w = floor(IMG_WIDTH * rn)
                crop_h = floor(IMG_HEIGHT * rn)
                w_space = IMG_WIDTH - crop_w
                h_space = IMG_HEIGHT - crop_h

                random_center_x = int(crop_w / 2) + random.randint(0, w_space)
                random_center_y = int(crop_h / 2) + random.randint(0, h_space)

                # Crop according to random parameters and resize to desired size
                input_image = input_image[random_center_y - int(crop_h / 2):random_center_y + int(crop_h / 2),
                                          random_center_x - int(crop_w / 2):random_center_x + int(crop_w / 2)]
                input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv.INTER_CUBIC)
                output_image = output_image[random_center_y - int(crop_h / 2):random_center_y + int(crop_h / 2),
                                            random_center_x - int(crop_w / 2):random_center_x + int(crop_w / 2)]
                output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv.INTER_NEAREST)

                # Adjust corner coordinates (adjust these calculations as needed)
                cornerTL_x = (cornerTL_x - (random_center_x - (IMG_WIDTH / 2))) * rn
                cornerTL_y = (cornerTL_y - (random_center_y - (IMG_HEIGHT / 2))) * rn
                cornerTR_x = (cornerTR_x - (random_center_x - (IMG_WIDTH / 2))) * rn
                cornerTR_y = (cornerTR_y - (random_center_y - (IMG_HEIGHT / 2))) * rn
                cornerBL_x = (cornerBL_x - (random_center_x - (IMG_WIDTH / 2))) * rn
                cornerBL_y = (cornerBL_y - (random_center_y - (IMG_HEIGHT / 2))) * rn
                cornerBR_x = (cornerBR_x - (random_center_x - (IMG_WIDTH / 2))) * rn
                cornerBR_y = (cornerBR_y - (random_center_y - (IMG_HEIGHT / 2))) * rn

            else:
                input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv.INTER_CUBIC)
                output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv.INTER_CUBIC)

            # ====== FLIP OR NOT? ====== #
            if R[1] < 0.5:
                input_image = cv.flip(input_image, 1)
                output_image = cv.flip(output_image, 1)
                cornerTL_x = IMG_WIDTH - cornerTL_x
                cornerTR_x = IMG_WIDTH - cornerTR_x
                cornerBL_x = IMG_WIDTH - cornerBL_x
                cornerBR_x = IMG_WIDTH - cornerBR_x

            if R[2] < 0.5:
                input_image = cv.flip(input_image, 0)
                output_image = cv.flip(output_image, 0)
                cornerTL_y = IMG_HEIGHT - cornerTL_y
                cornerTR_y = IMG_HEIGHT - cornerTR_y
                cornerBL_y = IMG_HEIGHT - cornerBL_y
                cornerBR_y = IMG_HEIGHT - cornerBR_y

            if R[3] < 0.3:
                seq = iaa.Sequential([iaa.MultiplyHueAndSaturation((0.8, 1.2), per_channel=True)])
                input_image = seq.augment_image(input_image)
                output_image = seq.augment_image(output_image)

            # Convert pixel range from [0,255] to [0,1]
            input_image = input_image / 255.0
            output_image = output_image / 255.0

            sorted_x = np.sort((cornerTL_x, cornerBL_x, cornerTR_x, cornerBR_x))
            sorted_y = np.sort((cornerTL_y, cornerBL_y, cornerTR_y, cornerBR_y))
            min_x = int(sorted_x[1]) + 10
            max_x = int(sorted_x[-2]) - 10
            min_y = int(sorted_y[1]) + 10
            max_y = int(sorted_y[-2]) - 10

            cv.rectangle(output_image, (0, 0), (IMG_WIDTH, min_y), (0, 0, 0), -1)
            cv.rectangle(output_image, (0, max_y), (IMG_WIDTH, IMG_HEIGHT), (0, 0, 0), -1)
            cv.rectangle(output_image, (0, 0), (min_x, IMG_HEIGHT), (0, 0, 0), -1)
            cv.rectangle(output_image, (max_x, 0), (IMG_WIDTH, IMG_HEIGHT), (0, 0, 0), -1)

            # (Optional debugging display omitted if not needed)

            # Append processed images and adjusted corners to lists
            input_images.append(input_image)
            output_images.append(output_image)
            batch_corners.append((cornerTL_x, cornerTL_y,
                                  cornerTR_x, cornerTR_y,
                                  cornerBL_x, cornerBL_y,
                                  cornerBR_x, cornerBR_y))

        except Exception as e:
            # If an error occurs processing an image, skip to the next image.
            continue

    # If no valid images were processed, return None for batch inputs/outputs
    if len(input_images) == 0:
        return None, None, []

    # Convert lists to torch tensors and rearrange dimensions to [B, C, H, W]
    input_images_tensor = torch.Tensor(np.array(input_images)).permute(0, 3, 1, 2)
    output_images_tensor = torch.Tensor(np.array(output_images)).permute(0, 3, 1, 2)
    return input_images_tensor.cuda(), output_images_tensor.cuda(), batch_corners





def yield_training_batch_binary(img_file_list, corners_list):

    batch_size = len(img_file_list)

    input_images = []
    output_images = []
    batch_corners = []

    for f in range(0,batch_size):

        # Read image from list and convert to array
        input_image_path = SOURCE_FOLDER+'/Lens_Dataset/INPUTS/'+str(img_file_list[f])+'.png'
        output_image_path = SOURCE_FOLDER+'/Lens_Dataset/OUTPUTS/'+str(img_file_list[f])+'.png'

        input_image = cv.imread(input_image_path)
        output_image = cv.imread(output_image_path)

        cornerTL_x = corners_list[f][0]
        cornerTL_y = corners_list[f][1]
        cornerTR_x = corners_list[f][2]
        cornerTR_y = corners_list[f][3]
        cornerBL_x = corners_list[f][4]
        cornerBL_y = corners_list[f][5]
        cornerBR_x = corners_list[f][6]
        cornerBR_y = corners_list[f][7]


        if DEBUG:
            debug_img = output_image.copy()
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),6,(255,255,255),-1)

            cv.imshow('PRE DEBUG OUTPT',  cv.resize(debug_img,(960,720)))


        #generate random vector to determine which augmentations to perform
        R = []
        for r in range(0,7):
            if DEBUG:
                R.append(0)
            else:
                rn = random.random()
                R.append(rn)

        #======ZOOM AND CROP OR NOT?======#

        if R[0] < 0.3:

            aspect_ratio = input_image.shape[1]/input_image.shape[0]
            target_ratio = IMG_WIDTH/IMG_HEIGHT
            height_center = int(input_image.shape[0]/2)
            width_center = int(input_image.shape[1]/2)

            if aspect_ratio > IMG_WIDTH/IMG_HEIGHT:
                target_width = int(input_image.shape[0]*target_ratio)
                input_image = input_image[0:input_image.shape[0], width_center-floor(target_width/2):width_center+floor(target_width/2)]
                output_image = output_image[0:input_image.shape[0], width_center-floor(target_width/2):width_center+floor(target_width/2)]

            elif aspect_ratio < IMG_WIDTH/IMG_HEIGHT:
                target_height = int(input_image.shape[1]*target_ratio)
                input_image = input_image[height_center-floor(target_height/2):height_center+floor(target_height/2), 0:input_image.shape[1]]
                output_image = output_image[height_center-floor(target_height/2):height_center+floor(target_height/2), 0:input_image.shape[1]]

            rn = random.uniform(1,1)    #how much to zoom in

            crop_w = floor(IMG_WIDTH*rn)    #how much to crop
            crop_h = floor(IMG_HEIGHT*rn)

            w_space = IMG_WIDTH - crop_w    #how much space is left
            h_space = IMG_HEIGHT - crop_h

            random_center_x = int(crop_w/2) + random.randint(0,w_space)     #center of the zoom/crop
            random_center_y = int(crop_h/2) + random.randint(0,h_space)

            #crops according to random parameters and resizes to desired input/output size
            input_image = input_image[random_center_y-int(crop_h/2):random_center_y+int(crop_h/2),random_center_x-int(crop_w/2):random_center_x+int(crop_w/2)]
            input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)
            output_image = output_image[random_center_y-int(crop_h/2):random_center_y+int(crop_h/2),random_center_x-int(crop_w/2):random_center_x+int(crop_w/2)]
            output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_NEAREST)


            # TODO: estas contas provavelmente estão mal, pelo que 'desabilitei o random crop e zoom'
            cornerTL_x = (cornerTL_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerTL_y = (cornerTL_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerTR_x = (cornerTR_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerTR_y = (cornerTR_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerBL_x = (cornerBL_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerBL_y = (cornerBL_y-(random_center_y - (IMG_HEIGHT/2)))*rn
            cornerBR_x = (cornerBR_x-(random_center_x - (IMG_WIDTH/2)))*rn
            cornerBR_y = (cornerBR_y-(random_center_y - (IMG_HEIGHT/2)))*rn

        else:

            input_image = cv.resize(input_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)
            output_image = cv.resize(output_image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv.INTER_CUBIC)



        #======FLIP OR NOT?======#

        #Flip Horizontaly
        if R[1] < 0.5:
            input_image = cv.flip(input_image, 1)
            output_image = cv.flip(output_image, 1)
            cornerTL_x = (IMG_WIDTH) - cornerTL_x
            cornerTR_x = (IMG_WIDTH) - cornerTR_x
            cornerBL_x = (IMG_WIDTH) - cornerBL_x
            cornerBR_x = (IMG_WIDTH) - cornerBR_x

        #Flip Verticaly
        if R[2] < 0.5:
            input_image = cv.flip(input_image, 0)
            output_image = cv.flip(output_image, 0)
            cornerTL_y = (IMG_HEIGHT) - cornerTL_y
            cornerTR_y = (IMG_HEIGHT) - cornerTR_y
            cornerBL_y = (IMG_HEIGHT) - cornerBL_y
            cornerBR_y = (IMG_HEIGHT) - cornerBR_y

        #Color Shift
        if R[3] < 0.3:
            seq = iaa.Sequential([iaa.MultiplyHueAndSaturation((0.8, 1.2), per_channel=True)])
            input_image = seq.augment_image(input_image)
            output_image = seq.augment_image(output_image)

        #Random Simplex Noise Blobs
        # if R[4] < 0.3:
        #     seq = iaa.SimplexNoiseAlpha( first=iaa.Multiply(mul = (0.6,1.4),per_channel=True),per_channel=True)
        #     input_image = seq.augment_image(input_image)
        #     output_image = seq.augment_image(output_image)

        #Now we have to work in [0,1] instead of [0,255]
        input_image = input_image/255.0
        output_image = output_image/255.0

        # Gaussian Blur
        # if R[5] < 0.3:
        #     input_image = cv.GaussianBlur(input_image,(5,5),0)

        #Gaussian Noise
        # if R[6] < 0:#0.2:
        #     mean = 0
        #     var = 0.001
        #     sigma = var**0.5
        #     gauss = np.random.normal(mean,sigma,(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS))
        #     gauss = gauss.reshape(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
        #     input_image = input_image + gauss

        sorted_x = np.sort((cornerTL_x,cornerBL_x,cornerTR_x,cornerBR_x))
        sorted_y = np.sort((cornerTL_y,cornerBL_y,cornerTR_y,cornerBR_y))

        # find the second largest/smallest of the x and y values, and give some manouvering margin to ensure the are no edges
        min_x = int(sorted_x[1])  + 10
        max_x = int(sorted_x[-2]) - 10
        min_y = int(sorted_y[1])  + 10
        max_y = int(sorted_y[-2]) - 10

        #min_x = int(min(cornerTL_x,cornerBL_x,cornerTR_x,cornerBR_x))
        #max_x = int(max(cornerTL_x,cornerBL_x,cornerTR_x,cornerBR_x))
        #min_y = int(min(cornerTL_y,cornerBL_y,cornerTR_y,cornerBR_y))
        #max_y = int(max(cornerTL_y,cornerBL_y,cornerTR_y,cornerBR_y))

        cv.rectangle(output_image,(0,0),(IMG_WIDTH,IMG_HEIGHT),(255,255,255),-1)
        cv.rectangle(output_image,(0,0),(IMG_WIDTH,min_y),(0,0,0),-1)
        cv.rectangle(output_image,(0,max_y),(IMG_WIDTH,IMG_HEIGHT),(0,0,0),-1)
        cv.rectangle(output_image,(0,0),(min_x,IMG_HEIGHT),(0,0,0),-1)
        cv.rectangle(output_image,(max_x,0),(IMG_WIDTH,IMG_HEIGHT),(0,0,0),-1)


        if DEBUG:
            debug_img = output_image.copy()
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTL_x),int(cornerTL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerTR_x),int(cornerTR_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBL_x),int(cornerBL_y)),6,(255,255,255),-1)

            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),10,(0,0,0),-1)
            cv.circle(debug_img,(int(cornerBR_x),int(cornerBR_y)),6,(255,255,255),-1)

            cv.imshow('DEBUG OUTPT',  cv.resize(debug_img,(960,720)))
            cv.waitKey()

        output_image = cv.cvtColor(output_image.astype(np.uint8),cv.COLOR_BGR2GRAY)
        output_image = np.expand_dims(output_image,axis=2)

        input_images.append(input_image)
        output_images.append(output_image)
        batch_corners.append((cornerTL_x,cornerTL_y,cornerTR_x,cornerTR_y,cornerBL_x,cornerBL_y,cornerBR_x,cornerBR_y))

    input_images_np  = np.stack(input_images,  axis=0)
    output_images_np = np.stack(output_images, axis=0)

    if input_images_np.ndim != 4:
      print("Skipping image with ID {img_file_list[f]} because one or both files were not found.")

    else:
      input_tensor  = torch.from_numpy(input_np).permute(0, 3, 1, 2).cuda()
      output_tensor = torch.from_numpy(output_np).permute(0, 3, 1, 2).cuda()


    return input_images.cuda(), output_images.cuda(), batch_corners

class EDSR(torch.nn.Module):
    def __init__(self, conv=common.default_conv):
        super(EDSR, self).__init__()

        rgb_range = 1.0
        n_resblocks = 5
        if DNN_TYPE==0:
            n_feats=3
            n_output_feats=3
            self.n_colors = 3
        else:
            n_feats=1
            n_output_feats=1
            self.n_colors = 3

        kernel_size = 3
        scale = 1
        act = torch.nn.ReLU(True)
        self.url = None
        self.sub_mean = common.MeanShift(rgb_range)
        self.add_mean = common.MeanShift(rgb_range, sign=1)

        self.res_scale = 1

        # define head module
        m_head = [conv(self.n_colors, n_feats, kernel_size)]

        # define body module
        m_body = [
            common.ResBlock(
                conv, n_feats, kernel_size, act=act, res_scale=self.res_scale
            ) for _ in range(n_resblocks)
        ]
        m_body.append(conv(n_feats, n_feats, kernel_size))

        # define tail module
        m_tail = [
            common.Upsampler(conv, scale, n_output_feats, act=False),
            conv(n_feats, self.n_colors, kernel_size)
        ]

        self.head = torch.nn.Sequential(*m_head)
        self.body = torch.nn.Sequential(*m_body)
        self.tail = torch.nn.Sequential(*m_tail)

    def forward(self, x):
        x = self.sub_mean(x)
        x = self.head(x)

        res = self.body(x)
        res += x

        #x = self.tail(res)
        x = self.add_mean(res)

        return x

    def load_state_dict(self, state_dict, strict=True):
        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name in own_state:
                if isinstance(param, torch.nn.Parameter):
                    param = param.data
                try:
                    own_state[name].copy_(param)
                except Exception:
                    if name.find('tail') == -1:
                        raise RuntimeError('While copying the parameter named {}, '
                                           'whose dimensions in the model are {} and '
                                           'whose dimensions in the checkpoint are {}.'
                                           .format(name, own_state[name].size(), param.size()))
            elif strict:
                if name.find('tail') == -1:
                    raise KeyError('unexpected key "{}" in state_dict'
                                   .format(name))

class EDSR_binary(torch.nn.Module):
    def __init__(self, conv=common.default_conv):
        super(EDSR_binary, self).__init__()

        rgb_range = 1.0
        n_resblocks = 5
        if DNN_TYPE==0:
            n_feats=3
            n_output_feats=3
            self.n_colors = 3
        else:
            n_feats=1
            n_output_feats=1
            self.n_colors = 3

        kernel_size = 3
        scale = 1
        act = torch.nn.ReLU(True)
        self.url = None
        self.sub_mean = common.MeanShift(rgb_range)
        self.add_mean = common.MeanShift(rgb_range, sign=1)
        self.collapse = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=1)

        self.res_scale = 1

        # define head module
        m_head = [conv(self.n_colors, n_feats, kernel_size)]

        # define body module
        m_body = [
            common.ResBlock(
                conv, n_feats, kernel_size, act=act, res_scale=self.res_scale
            ) for _ in range(n_resblocks)
        ]
        m_body.append(conv(n_feats, n_feats, kernel_size))

        # define tail module
        m_tail = [
            common.Upsampler(conv, scale, n_output_feats, act=False),
            conv(n_feats, self.n_colors, kernel_size)
        ]

        self.head = torch.nn.Sequential(*m_head)
        self.body = torch.nn.Sequential(*m_body)
        self.tail = torch.nn.Sequential(*m_tail)

    def forward(self, x):
        x = self.sub_mean(x)
        x = self.head(x)

        res = self.body(x)
        res += x

        #x = self.tail(res)
        x = self.add_mean(res)
        x=self.collapse(x)

        return x

    def load_state_dict(self, state_dict, strict=True):
        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name in own_state:
                if isinstance(param, torch.nn.Parameter):
                    param = param.data
                try:
                    own_state[name].copy_(param)
                except Exception:
                    if name.find('tail') == -1:
                        raise RuntimeError('While copying the parameter named {}, '
                                           'whose dimensions in the model are {} and '
                                           'whose dimensions in the checkpoint are {}.'
                                           .format(name, own_state[name].size(), param.size()))
            elif strict:
                if name.find('tail') == -1:
                    raise KeyError('unexpected key "{}" in state_dict'
                                   .format(name))

FILTER_SIZE = 128
PADDING = 1

class DoubleConv(torch.nn.Module):
    """(Conv => BN => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(inplace=True),

            torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[4,8, 16, 32]):
        super(UNet, self).__init__()

        # Downsampling path
        self.downs = torch.nn.ModuleList()
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Bottleneck
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)

        # Upsampling path
        self.ups = torch.nn.ModuleList()
        self.upconvs = torch.nn.ModuleList()

        rev_features = features[::-1]
        for feature in rev_features:
            self.upconvs.append(torch.nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature * 2, feature))

        # Final output layer
        if DNN_TYPE==0:
            self.final_conv = torch.nn.Conv2d(features[0], 3, kernel_size=1)
        else:
            self.final_conv = torch.nn.Conv2d(features[0], 1, kernel_size=1)


    def forward(self, x):
        # Encoder / Down path
        x1 = self.downs[0](x)              # [B, 64, H, W]
        x2 = self.downs[1](self.pool(x1))  # [B, 128, H/2, W/2]
        x3 = self.downs[2](self.pool(x2))  # [B, 256, H/4, W/4]
        x4 = self.downs[3](self.pool(x3))  # [B, 512, H/8, W/8]

        # Bottleneck
        x5 = self.bottleneck(self.pool(x4))  # [B, 1024, H/16, W/16]

        # Decoder / Up path
        u4 = self.upconvs[0](x5)
        if u4.shape != x4.shape:
            u4 = torch.nn.functional.interpolate(u4, size=x4.shape[2:])
        u4 = self.ups[0](torch.cat([x4, u4], dim=1))

        u3 = self.upconvs[1](u4)
        if u3.shape != x3.shape:
            u3 = torch.nn.functional.interpolate(u3, size=x3.shape[2:])
        u3 = self.ups[1](torch.cat([x3, u3], dim=1))

        u2 = self.upconvs[2](u3)
        if u2.shape != x2.shape:
            u2 = torch.nn.functional.interpolate(u2, size=x2.shape[2:])
        u2 = self.ups[2](torch.cat([x2, u2], dim=1))

        u1 = self.upconvs[3](u2)
        if u1.shape != x1.shape:
            u1 = torch.nn.functional.interpolate(u1, size=x1.shape[2:])
        u1 = self.ups[3](torch.cat([x1, u1], dim=1))

        # Final output
        return self.final_conv(u1)


#======================================== LOSS FUNCTION ==============================================

if DNN_TYPE==0:
    if TRAIN_TYPE==3:
        model=EDSR_binary()
    else:
        model=EDSR()
else:
    model=UNet()

def _gaussian_window(window_size: int, sigma: float) -> torch.Tensor:
    coords = torch.arange(window_size, dtype=torch.float)
    coords -= (window_size - 1) / 2.0
    g = torch.exp(-(coords ** 2) / (2 * sigma ** 2))
    g /= g.sum()
    return g

def _create_window(window_size: int, channel: int, device, dtype) -> torch.Tensor:
    # 1D Gaussian
    _1d = _gaussian_window(window_size, sigma=1.5).to(device=device, dtype=dtype)
    _2d = _1d.unsqueeze(1) @ _1d.unsqueeze(0)              # outer product → 2D kernel
    _2d = _2d.unsqueeze(0).unsqueeze(0)                     # shape (1,1,ws,ws)
    window = _2d.expand(channel, 1, window_size, window_size).contiguous()
    return window

class SSIM_Loss(nn.Module):
    """
    Structural Similarity (SSIM) loss.
    Returns 1 - mean(SSIM) over the batch.
    """

    def __init__(self, window_size: int = 11, size_average: bool = True):
        super().__init__()
        self.window_size = window_size
        self.size_average = size_average

    def forward(self, img1: torch.Tensor, img2: torch.Tensor) -> torch.Tensor:
        """Compute SSIM loss between two batches of images [B×C×H×W]."""

        # constants for stability (L = 1.0 since inputs are normalized to [0,1])
        C1 = (0.01 ** 2)
        C2 = (0.03 ** 2)

        batch, channel, height, width = img1.shape
        # make Gaussian window
        window = _create_window(self.window_size, channel, img1.device, img1.dtype)

        # compute means via depthwise conv
        mu1 = F.conv2d(img1, window, padding=self.window_size//2, groups=channel)
        mu2 = F.conv2d(img2, window, padding=self.window_size//2, groups=channel)

        mu1_sq = mu1.pow(2)
        mu2_sq = mu2.pow(2)
        mu1_mu2 = mu1 * mu2

        # compute variances & covariance
        sigma1_sq = F.conv2d(img1 * img1, window, padding=self.window_size//2, groups=channel) - mu1_sq
        sigma2_sq = F.conv2d(img2 * img2, window, padding=self.window_size//2, groups=channel) - mu2_sq
        sigma12   = F.conv2d(img1 * img2, window, padding=self.window_size//2, groups=channel) - mu1_mu2

        # SSIM map
        num = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
        den = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
        ssim_map = num / (den + 1e-12)

        if self.size_average:
            ssim_val = ssim_map.mean()
        else:
            # mean per image in batch
            ssim_val = ssim_map.view(batch, -1).mean(dim=1)

        # loss = 1 – SSIM
        return 1 - ssim_val


class MSE_Crop_Loss(torch.nn.Module):
    def __init__(self):
        super(MSE_Crop_Loss, self).__init__()
    def forward(self,pred, target, corners):

        crop_score = 0

        for b in range(len(corners)):
            corners_x = corners[b][0:8:2]
            corners_y = corners[b][1:8:2]
            min_x = min(corners_x)
            max_x = max(corners_x)
            min_y = min(corners_y)
            max_y = max(corners_y)

            if min_x<0:
                min_x=0
            if min_y<0:
                min_y=0
            if max_x<0:
                max_x=0
            if max_y<0:
                max_y=0
            if min_x>IMG_WIDTH:
                min_x=IMG_WIDTH
            if min_y>IMG_HEIGHT:
                min_y=IMG_HEIGHT
            if max_x>IMG_WIDTH:
                max_x=IMG_WIDTH
            if max_y>IMG_HEIGHT:
                max_y=IMG_HEIGHT

            cropped_tgt = torchvision.transforms.functional.crop(target[b],int(min_y),int(min_x),int(max_y-min_y), int(max_x-min_x))
            cropped_pred = torchvision.transforms.functional.crop(pred[b],int(min_y),int(min_x),int(max_y-min_y), int(max_x-min_x))
            #cropped_tgt = target[b][min_x:max_x,min_y:max_y]
            #cropped_pred = pred[b][min_x:max_x,min_y:max_y]
            crop_score += torch.nn.functional.mse_loss(cropped_pred,cropped_tgt)
            #crop_score = np.square(np.subtract(cropped_pred,cropped_tgt)).mean()

        score = torch.nn.functional.mse_loss(pred,target)

        crop_score = crop_score / len(corners)
        return 1-(0.9*crop_score + 0.1*score)

x = torch.randn(1,3,1920, 1080)
y = model(x)

aux = y[0,:,:,:].cpu().detach().numpy()*255
aux = aux.astype(np.uint8)
aux = np.transpose(aux, (1,2,0))

make_dot(y.mean(), params=dict(model.named_parameters())).render('Debulr_Model', format="png")

model_image = cv.imread('Debulr_Model.png')
cv.imwrite(SOURCE_FOLDER+'/Results/Deblur_Model.png',model_image)

model= torch.nn.DataParallel(model)
model.to('cuda')

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

if TRAIN_TYPE == 0:
    loss_function = MSE_Crop_Loss()
elif TRAIN_TYPE==3:
    loss_function = torch.nn.BCEWithLogitsLoss(reduction='mean')
elif TRAIN_TYPE==4:
    loss_function = SSIM_Loss()
else:
    loss_function = torch.nn.MSELoss()

# Create Log file
with open(training_history_filename, "w") as csv_file:
    progress_string = 'EPOCH,LOSS,TRAIN_SCORE,VAL_SCORE/n'
    csv_file.write(progress_string)


best_train_score = 0.0

if TRAIN:

    for epoch in range(EPOCHS):

        #torch.cuda.empty_cache()

        #reset running loss
        running_loss = 0.0
        running_train_score = 0.0
        running_val_score = 0.0

        #shuffle data list
        train_df = train_df.sample(frac=1)    #shuffle dataframe

        csv_row = train_df.iloc[:]
        train_img_list = list(csv_row['ID'])
        train_corners_list = list(csv_row[['cornerTL_x','cornerTL_y','cornerTR_x','cornerTR_y','cornerBL_x','cornerBL_y','cornerBR_x','cornerBR_y']].to_numpy())

        # train batches ---------------------------------------------
        #iterate through batches
        for batch in range(0,len(train_img_list),BATCH_SIZE):

            if (batch+BATCH_SIZE<=len(train_img_list)):
                batch_img_list = train_img_list[batch:batch+BATCH_SIZE]
                batch_corners_list = train_corners_list[batch:batch+BATCH_SIZE]
            else:
                if len(train_img_list)==1:
                    batch_img_list = [train_img_list[0]]
                    batch_corners_list = [train_corners_list[0]]
                else:
                    batch_img_list = train_img_list[batch:-1]
                    batch_corners_list = train_corners_list[batch:-1]

            if TRAIN_TYPE == 1:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch_black_BG(batch_img_list, batch_corners_list)
            elif TRAIN_TYPE == 3:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch_binary(batch_img_list, batch_corners_list)
            else:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch(batch_img_list, batch_corners_list)

            # Skip batch if no images were processed (i.e. helper returned None or an empty batch)
            if batch_inputs is None or batch_inputs.shape[0] == 0:
                continue

            #print(torch.cuda.memory_summary(device=None, abbreviated=False))

            #read each image and augment
            batch_preds = model(batch_inputs)

            #reset the gradients
            optimizer.zero_grad()

            #claculate loss for batch
            if TRAIN_TYPE == 0:
                loss = loss_function(batch_preds, batch_outputs.cuda(),batch_corners)
            else:
                loss = loss_function(batch_preds, batch_outputs.cuda())


            #calculate score for batch
            train_score = 1-loss

            #backward pass
            loss.backward()

            #step the optimizer ...?
            optimizer.step()

            running_loss += loss
            running_train_score += train_score




        # validation batches ---------------------------------------------
        #shuffle data list
        val_df = val_df.sample(frac=1)    #shuffle dataframe

        csv_row = val_df.iloc[:]
        val_img_list = list(csv_row['ID'])
        val_corners_list = list(csv_row[['cornerTL_x','cornerTL_y','cornerTR_x','cornerTR_y','cornerBL_x','cornerBL_y','cornerBR_x','cornerBR_y']].to_numpy())

        #iterate through batches
        for batch in range(0,len(val_img_list),BATCH_SIZE):
            if (batch+BATCH_SIZE<len(val_img_list)):
                batch_img_list = val_img_list[batch:batch+BATCH_SIZE]
                batch_corners_list = val_corners_list[batch:batch+BATCH_SIZE]
            else:
                if len(val_img_list)==1:
                    batch_img_list = [val_img_list[0]]
                    batch_corners_list = [val_corners_list[0]]
                else:
                    batch_img_list = val_img_list[batch:-1]
                    batch_corners_list = val_corners_list[batch:-1]

            if TRAIN_TYPE == 1:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch_black_BG(batch_img_list, batch_corners_list)
            elif TRAIN_TYPE == 3:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch_binary(batch_img_list, batch_corners_list)
            else:
                batch_inputs, batch_outputs, batch_corners = yield_training_batch(batch_img_list, batch_corners_list)

            #read each image and augment
            batch_preds = model(batch_inputs)

            #calculate score for batch
            if TRAIN_TYPE == 0:
                val_score = 1-loss_function(batch_preds, batch_outputs.cuda(), batch_corners)
            else:
                val_score = 1-loss_function(batch_preds, batch_outputs.cuda())


            running_val_score += val_score#.item()

        # Writes progress to csv file
        with open(training_history_filename, "a") as csv_file:  # append mode
            progress_string = str(epoch)+','+str(running_loss.item())+','+str(running_train_score.item())+','+str(running_val_score.item())+'/n'
            csv_file.write(progress_string)

        print('=========================================================')
        print('Epoch '+ str(epoch))
        print('Loss ' + str(running_loss.item()))
        print('Train Score '+ str(running_train_score.item()))
        print('Validation Score '+ str(running_val_score.item()))
        print('=========================================================')


        if running_train_score.item()>best_train_score:
            model_name = 'Best_Deblurr_model'

            if DNN_TYPE==0:
                if TRAIN_TYPE==3:
                    model_to_save=EDSR_binary()
                else:
                    model_to_save=EDSR()
            else:
                model_to_save=UNet()

            model_to_save.load_state_dict(model.module.state_dict())
            model_to_save.to('cuda')
            model_scripted = torch.jit.script(model_to_save)
            model_scripted.save(SOURCE_FOLDER+'/Results/'+model_name+'.pt')
            #torch.save(model.module.state_dict(),SOURCE_FOLDER+'/LineFeatures_Results/'+model_name+'.pt')
            best_train_score=running_train_score.item()

print('DONE')


if TEST:
    # Load the best model from file (update path if necessary)
    model_path = os.path.join(SOURCE_FOLDER, 'Results', '16.04.2025.pt')
    model_instance = torch.load(model_path,weights_only=False)
    model_instance.to('cuda')
    model_instance.eval()
    test_dir = os.path.join(SOURCE_FOLDER, 'Lens_Dataset', 'INPUTS')  # Set your test directory
    filelist = [f for f in os.listdir(test_dir) if f.endswith('.png')]

    # Display predictions for up to 10 random test images
    for i in range(min(10, len(filelist))):
        r = random.randrange(0, len(filelist))
        frame = cv.imread(os.path.join(test_dir, filelist[r]))
        # Resize and normalize for network input
        frame = cv.resize(frame, (1920, 1080)) / 255.0
        frame_T = np.transpose(frame, (2, 0, 1))
        np_tensor = np.expand_dims(frame_T, axis=0).astype(np.float32)
        input_tensor = torch.tensor(np_tensor).cuda()
        with torch.no_grad():
            preds = model_instance(input_tensor)
        # For SSIM mode, apply sigmoid to force outputs in [0,1]
        if TRAIN_TYPE == 4:
            preds = torch.sigmoid(preds)
        aux = preds[0, :, :, :].cpu().detach().numpy() * 255
        aux = aux.astype(np.uint8)
        output_image = np.transpose(aux, (1, 2, 0))
        imshow_cv2(output_image, title=f"Prediction for {filelist[r]}", resize_factor=0.5)

Epoch 0
Loss 7.682298183441162
Train Score 14.317704200744629
Validation Score 1.8278212547302246
Epoch 1
Loss 7.416843891143799
Train Score 14.583157539367676
Validation Score 1.91563081741333
Epoch 2
Loss 7.202683925628662
Train Score 14.797316551208496
Validation Score 1.8325625658035278
Epoch 3
Loss 6.939722537994385
Train Score 15.060276985168457
Validation Score 1.8994220495224
Epoch 4
Loss 6.9605865478515625
Train Score 15.039414405822754
Validation Score 2.1288437843322754
Epoch 5
Loss 6.448334693908691
Train Score 15.551666259765625
Validation Score 1.8521783351898193
Epoch 6
Loss 6.561953067779541
Train Score 15.43804931640625
Validation Score 1.9245989322662354
Epoch 7
Loss 6.245787620544434
Train Score 15.75421142578125
Validation Score 1.922702670097351
Epoch 8
Loss 6.421470642089844
Train Score 15.578529357910156
Validation Score 1.8698700666427612


In [ ]:
!ls  /content/Deblur/Results/

16.04.2025.pt  Best_Deblurr_model.pt  Deblur_Model.png	log.csv  Pytorch_0.pt
